# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 4 2022 3:27PM,250110,10,0086144698,ISDIN Corporation,Executing
1,Nov 4 2022 3:24PM,250108,10,0086144704,ISDIN Corporation,Released
2,Nov 4 2022 3:24PM,250108,10,0086144773,ISDIN Corporation,Released
3,Nov 4 2022 3:24PM,250108,10,0086144774,ISDIN Corporation,Released
4,Nov 4 2022 3:24PM,250108,10,0086144815,ISDIN Corporation,Executing
5,Nov 4 2022 3:24PM,250108,10,0086145208,ISDIN Corporation,Executing
6,Nov 4 2022 2:43PM,250103,21,632612,"NBTY Global, Inc.",Released
7,Nov 4 2022 2:41PM,250102,21,632301,"NBTY Global, Inc.",Released
8,Nov 4 2022 2:39PM,250100,21,631716,"NBTY Global, Inc.",Executing
9,Nov 4 2022 2:08PM,250097,10,8733261-SD,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,250102,Released,1
14,250103,Released,1
15,250108,Executing,2
16,250108,Released,3
17,250110,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250100,NaN,1.0,NaN
250102,NaN,NaN,1.0
250103,NaN,NaN,1.0
250108,NaN,2.0,3.0
250110,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250034,0.0,1.0,0.0
250040,0.0,0.0,1.0
250048,0.0,0.0,1.0
250063,0.0,0.0,1.0
250070,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250034,0,1,0
250040,0,0,1
250048,0,0,1
250063,0,0,1
250070,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250034,0,1,0
1,250040,0,0,1
2,250048,0,0,1
3,250063,0,0,1
4,250070,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250034,,1,
1,250040,,,1
2,250048,,,1
3,250063,,,1
4,250070,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation
6,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc."
7,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc."
8,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc."
9,Nov 4 2022 2:08PM,250097,10,Eywa Pharma Inc.
10,Nov 4 2022 1:54PM,250092,10,ISDIN Corporation
30,Nov 4 2022 1:48PM,250079,10,Eywa Pharma Inc.
35,Nov 4 2022 1:41PM,250091,19,"Emersa Waterbox, LLC"
36,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation,,1,
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation,,2,3
2,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc.",,,1
3,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc.",,,1
4,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc.",,1,
5,Nov 4 2022 2:08PM,250097,10,Eywa Pharma Inc.,,,1
6,Nov 4 2022 1:54PM,250092,10,ISDIN Corporation,2,10,8
7,Nov 4 2022 1:48PM,250079,10,Eywa Pharma Inc.,,4,1
8,Nov 4 2022 1:41PM,250091,19,"Emersa Waterbox, LLC",,,1
9,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation,,1,
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation,3,2,
2,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc.",1,,
3,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc.",1,,
4,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc.",,1,
5,Nov 4 2022 2:08PM,250097,10,Eywa Pharma Inc.,1,,
6,Nov 4 2022 1:54PM,250092,10,ISDIN Corporation,8,10,2
7,Nov 4 2022 1:48PM,250079,10,Eywa Pharma Inc.,1,4,
8,Nov 4 2022 1:41PM,250091,19,"Emersa Waterbox, LLC",1,,
9,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation,,1,
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation,3,2,
2,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc.",1,,
3,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc.",1,,
4,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation,NaN,1.0,NaN
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation,3.0,2.0,NaN
2,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 4 2022 3:27PM,250110,10,ISDIN Corporation,0.0,1.0,0.0
1,Nov 4 2022 3:24PM,250108,10,ISDIN Corporation,3.0,2.0,0.0
2,Nov 4 2022 2:43PM,250103,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 4 2022 2:41PM,250102,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Nov 4 2022 2:39PM,250100,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2000667,16.0,17.0,2.0
16,250034,0.0,1.0,0.0
19,500131,2.0,0.0,0.0
21,750305,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2000667,16.0,17.0,2.0
1,16,250034,0.0,1.0,0.0
2,19,500131,2.0,0.0,0.0
3,21,750305,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,17.0,2.0
1,16,0.0,1.0,0.0
2,19,2.0,0.0,0.0
3,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,16,Released,0.0
2,19,Released,2.0
3,21,Released,2.0
4,10,Executing,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,2.0,0.0,0.0,0.0
Executing,17.0,1.0,0.0,1.0
Released,16.0,0.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,2.0,0.0,0.0,0.0
1,Executing,17.0,1.0,0.0,1.0
2,Released,16.0,0.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,2.0,0.0,0.0,0.0
1,Executing,17.0,1.0,0.0,1.0
2,Released,16.0,0.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()